<a href="https://colab.research.google.com/github/myDSMLProjects/Deep-Learning/blob/master/Real_or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahulvv","key":"9179faa82c7c88b962e8fd802e61a571"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 57.0MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.7MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 22.9MB/s]


In [4]:
import pandas as pd
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential


nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
df.text = df.text.astype(str)
test_df.text = test_df.text.astype(str)

In [10]:
import string
import re

#remove urls
def remove_urls(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)
  
#remove html tags
def remove_html(text):
  html=re.compile(r'<.*?>')
  return html.sub(r'',text)

# splitting the text
def split_text(text):
  text = text.split()
  return text

 # making lower case words
def lower(text):
  text = [word.lower() for word in text]
  return str(text)

#remove punct
def remove_punct(text):
  text = ''.join([char for char in text if char not in string.punctuation])
  text = re.sub('[0-9]+', '', str(text))
  return text

#remove stopwords
def remove_stopwords(text):
  pattern = re.compile(r'\b('+r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub(' ', text)
  return text

#lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  text = lemmatizer.lemmatize(text)
  return text

In [11]:
def final(text):
  
  t0 = remove_urls(text)
  t1 = remove_html(t0)
  t2 = split_text(t1)
  t3 = lower(t2)
  t4 = remove_punct(t3)
  t5 = remove_stopwords(t4)
  t6 = lemmatize_words(t5)
  return t6

In [12]:
X = df.text.to_list()
X_processed = []
for i in range(len(X)):
  data = final(X[i])
  X_processed.append(data)

X = X_processed
y = df.target

X_test = test_df.text.to_list()
X_test_processed = []
for i in range(len(X_test)):
  data = final(X_test[i])
  X_test_processed.append(data)

X_test = X_test_processed

In [13]:
vocab_size = 17096
embedding_dim=100
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = 'OOV'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(X)
padded = pad_sequences(sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

training_labels = np.array(y)

In [14]:
print(len(tokenizer.word_index))

17095


In [15]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(padded, training_labels, test_size=0.33, random_state=42)

In [16]:
from tensorflow.keras.regularizers import l2
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001))),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1709600   
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 128)           84480     
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dropout (Dropout)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 24)                153624    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 1,947,729
Trainable params: 1,947,729
Non-trainable params: 0
______________________________________________

In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
optimzer=Adam(learning_rate=1e-3)
model.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])
num_epochs=30

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

model.fit(X_tr, y_tr, validation_data = (X_te, y_te), epochs=num_epochs, callbacks=[checkpoint])

Epoch 1/30
160/160 [==============================] - 5s 28ms/step - loss: 0.6927 - accuracy: 0.7061 - val_loss: 0.4900 - val_accuracy: 0.7990
Epoch 2/30
160/160 [==============================] - 4s 23ms/step - loss: 0.3040 - accuracy: 0.8941 - val_loss: 0.5482 - val_accuracy: 0.7668
Epoch 3/30
160/160 [==============================] - 4s 22ms/step - loss: 0.1609 - accuracy: 0.9514 - val_loss: 0.5462 - val_accuracy: 0.7684
Epoch 4/30
160/160 [==============================] - 4s 22ms/step - loss: 0.1100 - accuracy: 0.9702 - val_loss: 0.6793 - val_accuracy: 0.7612
Epoch 5/30
160/160 [==============================] - 4s 22ms/step - loss: 0.0888 - accuracy: 0.9745 - val_loss: 0.6682 - val_accuracy: 0.7688
Epoch 6/30
160/160 [==============================] - 4s 23ms/step - loss: 0.0771 - accuracy: 0.9794 - val_loss: 0.6552 - val_accuracy: 0.7859
Epoch 7/30
160/160 [==============================] - 4s 22ms/step - loss: 0.0653 - accuracy: 0.9810 - val_loss: 0.7227 - val_accuracy: 0.7565

In [19]:
model.load_weights('model.h5')
pred = model.predict(testing_padded)

In [20]:
test_df.shape

(3263, 4)

In [21]:
pred

array([[0.71211314],
       [0.8382095 ],
       [0.92938125],
       ...,
       [0.84488654],
       [0.94368005],
       [0.6653006 ]], dtype=float32)

In [22]:
pred = (np.round(pred)).astype(int)
pred = pred.reshape(3263)

In [23]:
pd.read_csv('sample_submission.csv')

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [24]:
my_submission = pd.DataFrame({'id': test_df.id, 'target': pred})
my_submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [25]:
my_submission.to_csv('submission.csv', index=False)

In [26]:
#!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "This is my first submission"

100% 22.2k/22.2k [00:06<00:00, 3.77kB/s]
403 - Your team has used its submission allowance (5 of 5). This resets at midnight UTC (12 hours from now).
